# Import Library 

In [1]:
import pandas
import configparser
import psycopg2

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['db']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

In [3]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

# Create House Table 

In [12]:
table_sql = """
            CREATE TABLE IF NOT EXISTS gp23.house
            (

                price integer,
                bed integer,
                bath integer,
                area integer,
                address VARCHAR(200),
                PRIMARY KEY(address)
            );

            """

In [13]:
conn.rollback()


In [14]:

cur.execute(table_sql)
conn.commit()

# Define the Table 

In [24]:

url = 'https://www.trulia.com/VA/Ashburn/20147/'

In [26]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
print(html_data.decode('utf-8'))

<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><script>
            window.__uspapi = function(command, version, callback) {
              try {
                if (command === 'getUSPData') {
                  var cookies = document.cookie.split(';');
                  for (var i = 0; i < cookies.length; i++) {
                    var cookie = cookies[i];
                    var separatorIndex = cookie.indexOf('=');
                    separatorIndex = separatorIndex < 0 ? cookie.length : separatorIndex;
                    var cookie_name = decodeURIComponent(cookie.slice(0, separatorIndex).replace(/^\s+/, ''));
                    if (cookie_name === 'usprivacy') {
                      var uspString = decodeURIComponent(cookie.slice(separatorIndex + 1));
                      callback({ version: version, uspString: uspString }, true);
                      return;
                    }
                  }
                }
              } catch (ex) {
                c

In [27]:

from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
print (soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><script>
            window.__uspapi = function(command, version, callback) {
              try {
                if (command === 'getUSPData') {
                  var cookies = document.cookie.split(';');
                  for (var i = 0; i < cookies.length; i++) {
                    var cookie = cookies[i];
                    var separatorIndex = cookie.indexOf('=');
                    separatorIndex = separatorIndex < 0 ? cookie.length : separatorIndex;
                    var cookie_name = decodeURIComponent(cookie.slice(0, separatorIndex).replace(/^\s+/, ''));
                    if (cookie_name === 'usprivacy') {
                      var uspString = decodeURIComponent(cookie.slice(separatorIndex + 1));
                      callback({ version: version, uspString: uspString }, true);
                      return;
                    }
                  }
                }
              } catch (ex) {
                

In [30]:
for li_class in soup.find_all('li', class_ = 'Grid__CellBox-sc-144isrp-0 SearchResultsList__WideCell-b7y9ki-2 jiZmPM'):
    try:
        for price_div in li_class.find_all('div',{'data-testid':'property-price'}):
            price =int(price_div.text.replace('$','').replace(",",""))
        for bed_div in li_class.find_all('div', {'data-testid':'property-beds'}):
            bed= int(bed_div.text.replace('bd','').replace(",",""))
        for bath_div in li_class.find_all('div',{'data-testid':'property-baths'}):
            bath =int(bath_div.text.replace('ba','').replace(",",""))
        for area_div in li_class.find_all('div',{'data-testid':'property-floorSpace'}):
            area=int(area_div.text.split('sqft')[0].replace(",",""))
        for address_div in li_class.find_all('div',{'data-testid':'property-address'}):
            address =address_div.text

        try:
            sql_insert = """
                        insert into gp23.house(price,bed,bath,area,address)
                        values('{}','{}','{}','{}','{}')            
                        """.format(price,bed,bath,area,address)

            cur.execute(sql_insert)
            conn.commit()
        except:
            conn.rollback()
    except:
        pass

# Query the Table

In [31]:
df = pandas.read_sql_query('select * from gp23.house ', conn)
df[:]

price  bed  bath  area                                           address
0    984900    5     4  4563             19952 Hazeltine Pl, Ashburn, VA 20147
1    349900    2     2  1598         43071 Old Gallivan Ter, Ashburn, VA 20147
2    699000    4     4  2824           20376 Charter Oak Dr, Ashburn, VA 20147
3    691025    4     5  3555           20445 Charter Oak Dr, Ashburn, VA 20147
4    775000    4     4  3570              20254 Ordinary Pl, Ashburn, VA 20147
5    550000    3     3  2118         44390 Oakmont Manor Sq, Ashburn, VA 20147
6    860000    5     4  3711           20879 Yellowbloom Ct, Ashburn, VA 20147
7    337000    3     2  1284  43840 Hickory Corner Ter #108, Ashburn, VA 20147
8    568000    3     3  2348             21610 Monmouth Ter, Ashburn, VA 20147
9    630000    3     3  1800               43308 Ardmore St, Ashburn, VA 20147
10   889000    5     4  4950               20890 Medinah Ct, Ashburn, VA 20147
11   839000    4     4  3927               20412 Peckham St, Ashburn, VA 20147
12   949900    4     4  4255            20634 Marblehead Dr, Ashburn, VA 20147
13   229900    2     2   876     20570 Hope Spring Ter #104, Ashburn, VA 20147
14   615000    3     3  2678              44732 Tiverton Sq, Ashburn, VA 20147
15   797000    3     3  2452              20431 Old Grey Pl, Ashburn, VA 20147
16   599950    4     4  3000              44262 Marchand Ln, Ashburn, VA 20147
17   319900    2     2  1184         44152 Natalie Ter #201, Ashburn, VA 20147
18   350000    2     2  1019    20979 Timber Ridge Ter #103, Ashburn, VA 20147
19   504900    3     4  1718            19885 Ridgecrest Sq, Ashburn, VA 20147
20   389000    2     2  1216                21799 Findon Ct, Ashburn, VA 20147
21   425000    3     3  1520           44248 Mossy Brook Sq, Ashburn, VA 20147
22   510000    3     4  1886             20241 Brookview Sq, Ashburn, VA 20147
23  1432000    5     3  4671          20721 Jennifer Ann Dr, Ashburn, VA 20147
24   502000    3     4  2035           21016 Coach House Sq, Ashburn, VA 20147
25   249900    1     1   862       20601 Cornstalk Ter #101, Ashburn, VA 20147
26   262000    2     1   877       20290 Beechwood Ter #303, Ashburn, VA 20147
27   239900    1     1   747    20977 Timber Ridge Ter #101, Ashburn, VA 20147
28   915950    3     4  3206           43879 Hanes Hall Ter, Ashburn, VA 20147
29  1275000    5     5  5963         20202 Kiawah Island Dr, Ashburn, VA 20147
30   793560    3     4  2385             20695 Glenmont Ter, Ashburn, VA 20147
31  6500000    3     4  2385              42649 Hearford Ln, Ashburn, VA 20147
32   632900    2     2  1474        44691 Wellfleet Dr #407, Ashburn, VA 20147
33   539899    3     4  1866                 43327 Foyt Ter, Ashburn, VA 20147
34   684990    3     4  2347                   Glenmont Ter, Ashburn, VA 20147

# Basic Stat

In [32]:
df.describe()

price        bed       bath         area
count  3.500000e+01  35.000000  35.000000    35.000000
mean   7.968993e+05   3.142857   3.200000  2482.000000
std    1.031834e+06   1.088519   1.106133  1327.351343
min    2.299000e+05   1.000000   1.000000   747.000000
25%    4.070000e+05   2.500000   2.000000  1497.000000
50%    6.150000e+05   3.000000   4.000000  2347.000000
75%    8.180000e+05   4.000000   4.000000  3380.500000
max    6.500000e+06   5.000000   5.000000  5963.000000

# Price Distribution 

In [33]:
df['price'].hist()

<AxesSubplot:>

# Bed vs Bath 

In [ ]:
df.plot.scatter(x='bed',y='bath')